# Euler–Maclaurin 2D formula (using halfsum)

In [1]:
def num_int(f, a, b, v):
    from scipy import integrate

    try:
        if logging:
            try:
                print ("v = ", v, "integrand = ", f(v))
            except Exception as ex2:
                print ("Exception ex2 = ", ex2)
            
        integral = integrate.quad(f, a, b)

        result = integral[0]
        return result

    except Exception as ex:

        print ("Exception ex = ", str(ex))
        print ("f = ", f)
        try:
            print ("v = ", v, "integrand = ", f(v))
        except Exception as ex2:
            print ("Exception ex2 = ", ex2)

        print ("a, b = ", a, ",", b)
        try:
            print ("f(a) = ", f(a))
        except:
            pass
        try:
            print ("f(b) = ", f(b))
        except:
            pass

        import inspect
        stack = inspect.stack()
        for frame in stack:
            func_name = frame[3]
            print ("func_name = ", func_name)

        if True or "unable to simplify to float approximation" == str(ex):
            from sage.symbolic.integration.integral import definite_integral
            return definite_integral(f(v), v, a, b, hold=True)

        raise ex

def subs(ex, *argv):
    if hasattr(ex, 'subs'):
        return ex.subs(*argv)
    return ex

def e2n(ex):
    if hasattr(ex, 'n'):
        return ex.n()
    return ex

In [2]:
logging = False

calc_R_p=True

def my_euler_maclaurin1(f,symb,a,b,p, f_diff_symb_p=None):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")


    sum_dfdx_a_bernoullis = sum([(diff(f,symb,2*k-1))*(bernoulli(2*k)/factorial(2*k)) for k in range(1,1+int(p/2))])
    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    if True:
        B = lambda x, p : bernoulli_polynomial(x, p)
        P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
        fact = lambda n : factorial(n)
    else:
        B = function("B")(x,p)
        P = function("P")(x,p)
        fact = function("fact")(n)
        
    if True == calc_R_p:
        # Can't calculate derivative wrt j + u.
        #int_R_p = lambda j : integral(f.diff(symb,p).subs(symb == u+j) * bernoulli_polynomial(u,p)/factorial(p), (u,0,1))
        def int_R_p(f,symb,j):
            v = var("v")            
            if None == f_diff_symb_p:
                deriv = diff(f,symb,p)
            else:
                deriv = f_diff_symb_p
            if logging:
                print("deriv", deriv)
            deriv_s = deriv.subs(symb == v+j)
            if logging:
                print("deriv_s", deriv_s)
            int_f_diff_B_fact = integral(deriv_s * bernoulli_polynomial(v,p)/factorial(p), (v,0,1))
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)
            return int_f_diff_B_fact
        
        def int_R_p_shifted_first(f,symb,j):
            v = var("v")
            fn_shifted = f.subs(symb == v + j)
            if logging:
                print("fn_shifted", fn_shifted)
            deriv = fn_shifted.diff(v,p)
            if logging:
                print("deriv", deriv)
            int_f_diff_B_fact = integral(deriv * B(x=v,p=p)/fact(n=p), (v,0,1))
    
            if logging:
                print("int_f_diff_B_fact", int_f_diff_B_fact)

            return int_f_diff_B_fact

        if logging:
            print("int_R_p(j = a)", int_R_p(f,symb,j = a))
            
        if use_int_R_p_j:
            v, j = var("v, j")
            assume(j>0)
            from sage.calculus.calculus import symbolic_sum
            if change_R_p_order:
                # Change order of summation and integration
                if use_int_R_p_shifted_first:
                    R_p = (-1)^(p+1)*integral(symbolic_sum(f.subs(symb == v+j).diff(v,p)*B(x=v,p=p)/fact(n=p), \
                                                  j, a, b-1), \
                                              (v,0,1))
                else:
                    if None == f_diff_symb_p:
                        f_diff_symb_p = diff(f, symb, p)
                    R_p = (-1)^(p+1)*integral(symbolic_sum(f_diff_symb_p.subs(symb == v+j)*B(x=v,p=p)/fact(n=p), \
                                                  j, a, b-1), \
                                              (v,0,1))
            else:
                if use_int_R_p_shifted_first:
                    R_p = (-1)^(p+1)*symbolic_sum(int_R_p_shifted_first(f,symb,j=j), j, a, b-1)
                else:
                    R_p = (-1)^(+1)*symbolic_sum(int_R_p(f,symb,j=j), j, a, b-1)
                
        else:
            if None == f_diff_symb_p:
                f_diff_symb_p = diff(f,symb,p)
            if not use_num_int:
                R_p = (-1)^(p+1)*integral(f_diff_symb_p * P(x=symb,p=p)/fact(n=p),(symb,a,b),hold=True)
            else:
                R_p = (-1)^(p+1)*num_int(lambda symb : f_diff_symb_p * P(x=symb,p=p)/fact(n=p), a, b, v=symb)
                

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
        
    sum_fa_fb_2 = (subs(f, symb == a) + subs(f, symb == b))/2
    
    s = int_fn_a_b + sum_fa_fb_2 - subs(sum_dfdx_a_bernoullis, symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    if True == calc_R_p:
        s += R_p
        
    if False:
        print("int_fn_a_b", int_fn_a_b)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis(a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis(b))
        if True == calc_R_p:
            print("int_R_p(j = a)", int_R_p(j = a))
            print("R_p", R_p)
        print("")
        print("s",s)
    return s

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+{\frac {f(a)+f(b)}{2}}+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{2k}}{(2k)!}}(f^{(2k-1)}(b)-f^{(2k-1)}(a))+R_{p},}$$

$${\displaystyle \sum _{i=a}^{b}f(i)=\int _{a}^{b}f(x)\,dx+\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(f^{(k-1)}(b)-f^{(k-1)}(a))+R_{p},}$$

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(v+j){\frac {B_{p}(v)}{p!}}\,dv.}$$

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$${\displaystyle R_{p}=(-1)^{p+1}\int _{a}^{b}f^{(p)}(x){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [3]:
# https://www.journals.vu.lt/LMJ/article/view/20600/19701
# ФОРМУЛА СУММИРОВАНИЯ ЭЙЛЕРА—МАКЛОРЕНА ДЛЯ ФУНКЦИИ МНОГИХ ПЕРЕМЕННЫХ А.БИКЯЛИС
# VIII LIETUVOS MATEMATIKOS RINKINYS 4
# ЛИТОВСКИЙ МАТЕМАТИЧЕСКИЙ СБОРНИК
# 1968

# Далее,обе стороны этого равенства суммируем по всем целым m2 из интервала(z2,y2). Получаем

if True:
    B = lambda x, p : bernoulli_polynomial(x, p)
    P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
    fact = lambda n : factorial(n)
else:
    B = function("B")(x,p)
    P = function("P")(x,p)
    fact = function("fact")(n)

def integral_R_p(f,symb,jx,hold, f_diff_symb_p=None):
    vx = var("v_"+str(symb))
    if None == f_diff_symb_p:
        deriv = diff(f, symb,p)
    else:
        deriv = f_diff_symb_p
    if logging:
        print("deriv", deriv)

    deriv_subs = deriv.subs(symb == vx+jx)
    if logging:
        print("deriv_subs", deriv_subs)

    int_f_diff_B_fact = integral(deriv_subs * bernoulli_polynomial(vx,p)/factorial(p), (vx,0,1), hold=hold)
    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact

def integral_R_p_shifted_first(f,symb,jx,hold):
    vx = var("v_"+str(symb))
    fn_shifted = f.subs(symb == vx + jx)
    if logging:
        print("fn_shifted", fn_shifted)

    deriv = fn_shifted.diff(vx,p)
    if logging:
        print("deriv", deriv)

    int_f_diff_B_fact = integral(deriv * B(x=vx,p=p)/fact(n=p), (vx, 0, 1), hold=hold)

    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact
    
    
def euler_maclaurin_R_p(f,symb,a,b,p, f_diff_symb_p=None, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        if f_diff_symb_p is not None:
            print("f_diff_symb_p", f_diff_symb_p)
        print("")
    if use_int_R_p_j:
        jx = var('j_'+str(symb))
        assume(jx>0)
        from sage.calculus.calculus import symbolic_sum
        
        if change_R_p_order:
            vx = var("v_"+str(symb))
            # Change order of summation and integration
            if use_int_R_p_shifted_first:
                R_p = (-1)^(p+1)*integral(symbolic_sum(f.subs(symb == vx+jx).diff(vx,p)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a, b-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int)
            else:
                if None == f_diff_symb_p:
                    f_diff_symb_p = diff(f,symb,p)
                R_p = (-1)^(p+1)*integral(symbolic_sum(f_diff_symb_p.subs(symb == vx+jx)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a, b-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int)
        else:       
            if use_int_R_p_shifted_first:
                term = (-1)^(p+1)*integral_R_p_shifted_first(f=f, symb=symb, jx=jx, hold=hold_int)
                R_p = symbolic_sum(term, jx, a, b-1, hold=hold_sum)
            else:
                term = (-1)^(p+1)*integral_R_p(f=f, symb=symb, jx=jx, hold=hold_int, f_diff_symb_p=f_diff_symb_p)
                R_p = symbolic_sum(term, jx, a, b-1, hold=hold_sum)
            return R_p, term
    else:
        if None == f_diff_symb_p:
            f_diff_symb_p = diff(f,symb,p)
        if not use_num_int:
            if logging:
                print("(-1)^(p+1)*P(x=symb,p=p)/fact(n=p)", (-1)^(p+1)*P(x=symb,p=p)/fact(n=p))
            R_p = (-1)^(p+1)*integral(f_diff_symb_p * P(x=symb,p=p)/fact(n=p),(symb,a,b),hold=hold_int)
        else:
            R_p = (-1)^(p+1)*num_int(lambda symb: f_diff_symb_p * P(x=symb,p=p)/fact(n=p), a, b, v=symb)
            
    return R_p

$$\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(f^{(k-1)}(b)-f^{(k-1)}(a))$$

In [4]:
def half_summa(f,symb,a,b):

    if logging:
        print("half_summa", f,symb, a, b)
        print("half_sum(a)", f.subs(symb == a)/2)
        if Infinity != b:
            print("half_sum(b)", f.subs(symb == b)/2)

    s = subs(f, symb == a)/2
    if Infinity != b:
        s += f.subs(symb == b)/2
    return s


In [5]:
def sum_dfdx_bernoulis(f,symb,a,b,p):
    dfdx_a_bernoullis = []
    for k in range(1,1+int(p/2)):
        dfdx_a_bernoullis += [(diff(f,symb,2*k-1))*(bernoulli(2*k)/factorial(2*k))]
        
    sum_dfdx_a_bernoullis = sum(dfdx_a_bernoullis)

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    s = - subs(sum_dfdx_a_bernoullis, symb == a)
    if Infinity != b:
        s += subs(sum_dfdx_a_bernoullis, symb == b)
    return s

In [6]:
def definite_integral_derivative(f, x, a, b, diff_param, hold=True):
    from sage.symbolic.integration.integral import definite_integral

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = definite_integral(f.diff(diff_param), x, a, b, hold=hold)
    else:
        ans = SR.zero()

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += f.subs(x == b) * b.diff(diff_param)
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans -= f.subs(x == a) * a.diff(diff_param)

    return ans


In [7]:
def definite_integral_derivative_str(f, x, a, b, diff_param=None, hold=False, algorithm=None):

    if not x.has(diff_param):
        # integration variable != differentiation variable
        if algorithm is not None:
            ans = "integrate((" + str(f) + ").diff(" + str(diff_param) + "), " + str(x) + ", " + str(a) + ", " + str(b) + ", algorithm = "\""+ algorithm + ", hold = " + str(hold) + ")"
        else:
            ans = "definite_integral((" + str(f) + ").diff(" + str(diff_param) + "), " + str(x) + ", " + str(a) + ", " + str(b) + ", hold = " + str(hold) + ")"
    else:
        ans = ""

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += " + " + str(f.subs(x == b) * b.diff(diff_param))
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans += " - " + str(f.subs(x == a) * a.diff(diff_param))

    return ans

In [8]:
def definite_num_int_derivative_str(f, x, a, b, diff_param=None):

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = "num_int(lambda " + str(x)+ " : " + str(f.diff(diff_param)) + ", " + str(a) + ", " + str(b) + ", v = " + str(x) + ")"
    else:
        ans = ""

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += " + " + str(f.subs(x == b) * b.diff(diff_param))
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans += " - " + str(f.subs(x == a) * a.diff(diff_param))

    return ans

$$\sum _{k=1}^{\lfloor p/2\rfloor }{\frac {B_{k}}{(k)!}}(g(y)^{(k-1)}(b_y)-g(y)^{(k-1)}(a_y))$$
where 

$$g(y) = {\displaystyle R_{p}(y)=(-1)^{p+1}\sum_{j=a_x}^{b_x-1} \int _{0}^{1}\frac{\partial^{p}}{\partial x^{p}}f(v_x+j,y){\frac {B_{p}(v_x)}{p!}}\,dv_x.}$$
or

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$$g(y) = {\displaystyle R_{p}=(-1)^{p+1}\int _{a_x}^{b_x}\frac{\partial^{p}}{\partial x^{p}}f(x,y){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [9]:
#int_R_p = integral(euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p), (y, a_y, b_y), hold=True)

def integral_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("f(x,y)", f(x=x,y=y))
        if None != f_diff_x_p:
            print("f_diff_x_p", f_diff_x_p)
            print("f_diff_x_p(x,y)", f_diff_x_p(x=x,y=y))
        print("")
    if use_int_R_p_j:
        jx = var('j_x')
        assume(jx>0)
        from sage.calculus.calculus import symbolic_sum
        
        if change_R_p_order:
            vx = var("v_"+str(x))
            # Change order of summation and integration
            if use_int_R_p_shifted_first:
                int_R_p = (-1)^(p+1)*integral(integral(symbolic_sum(f.subs(x == vx+jx).diff(vx,p)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a_x, b_x-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int), y, a_y, b_y, hold=hold_int)
            else:
                if None == f_diff_x_p:
                    f_diff_x_p = f.diff(x,p)
                int_R_p = (-1)^(p+1)*integral(integral(symbolic_sum(f_diff_x_p.subs(x == vx+jx)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a_x, b_x-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int), y, a_y, b_y, hold=hold_int)
        else:       
            if use_int_R_p_shifted_first:
                int_int = (-1)^(p+1)*integral(integral_R_p_shifted_first(f=f, symb=x, jx=jx, hold=hold_int), y, a_y, b_y, hold=hold_int)
                int_R_p = symbolic_sum(int_int, jx, a_x, b_x-1, hold=hold_sum)
            else:
                int_int = (-1)^(p+1)*integral(integral_R_p(f=f, symb=x, jx=jx, hold=hold_int), y, a_y, b_y, hold=hold_int)
                int_R_p = symbolic_sum(int_int, jx, a_x, b_x-1, hold=hold_sum)
            return (int_R_p, int_int)
    else:
        if None == f_diff_x_p:
            f_diff_x_p = f.diff(x,p)
        integrand = lambda x, y, p : f_diff_x_p(x=x,y=y) * P(x=x,p=p)/fact(n=p)
        if logging:
            print("integrand", integrand(x,y,p))
        if not use_num_int:
            int_R_p = (-1)^(p+1)*integral(integral(integrand(x,y,p),(x,a_x,b_x),hold=hold_int), y, a_y, b_y, hold=hold_int)
        else:
            int_R_p = (-1)^(p+1)*num_int(lambda y : num_int(lambda x : integrand(x,y,p), a_x, b_x, x), a_y, b_y, y)
            
    return int_R_p

$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a_y}^{b_y-1} \int _{0}^{1}g^{(p)}(v_y+j){\frac {B_{p}(v_y)}{p!}}\,dv_y.}$$

$${\displaystyle P_{k}(y)=B_{k}(y-\lfloor y\rfloor ),}$$

$${\displaystyle R_{p}=(-1)^{p+1}\int _{a_y}^{b_y}g^{(p)}(y){\frac {P_{p}(y)}{p!}}\,dy.}$$

where

$$g(y) = {\displaystyle R_{p}(y)=(-1)^{p+1}\sum_{j=a_x}^{b_x-1} \int _{0}^{1}\frac{\partial^{p}}{\partial x^{p}}f(v_x+j,y){\frac {B_{p}(v_x)}{p!}}\,dv_x.}$$
or

$${\displaystyle P_{k}(x)=B_{k}(x-\lfloor x\rfloor ),}$$

$$g(y) = {\displaystyle R_{p}=(-1)^{p+1}\int _{a_x}^{b_x}\frac{\partial^{p}}{\partial x^{p}}f(x,y){\frac {P_{p}(x)}{p!}}\,dx.}$$

In [10]:
#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def euler_maclaurin_R_p_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p, f_diff_x_p_diff_y_p):

    from sage.calculus.calculus import symbolic_sum
    from sage.symbolic.integration.integral import definite_integral
    if use_int_R_p_j:
        vx,vy = var("v_x, v_y")
        jx,jy = var("j_x, j_y")
        assume(v_y+j_y>0)
        
        if None == f_diff_x_p_diff_y_p:
            f_diff_x_p_diff_y_p = f.diff(x,p).diff(y,p)

        R_p_R_p = (-1)^(p+1)*symbolic_sum(definite_integral( \
                               (-1)^(p+1)*symbolic_sum(definite_integral(f_diff_x_p_diff_y_p.subs(x == vx + jx)  \
                                                              * B(x=vx,p=p)/fact(n=p), \
                                 vx,0,1,hold=True), v=jx, a=a_x, b=b_x-1,hold=True) \
                                                                        .subs(y == vy + jy)  \
                                                              * B(x=vy,p=p)/fact(n=p), \
                                 vy,0,1,hold=True), v=jy, a=a_y, b=b_y-1,hold=True)
    else:
        if not use_num_int:

            if None == f_diff_x_p_diff_y_p:
                f_diff_x_p_diff_y_p = f.diff(x,p)

            R_p_R_p = (-1)^(p+1)*definite_integral( \
                               (-1)^(p+1)*definite_integral(f_diff_x_p_diff_y_p  \
                                                              * P(x=x,p=p)/fact(n=p), \
                                 x, a_x, b_x, hold=True).diff(y,p)  \
                                                              * P(x=y,p=p)/fact(n=p), \
                                 y, a_y, b_y, hold=True)
        else:
            f_diff_x_p_diff_y_p_num_str = "f_diff_x_diff_y_num = lambda " + str(x) + ", " + str(y) + ":" + str(f(x,y).diff(x,p).diff(y,p))
            print("f_diff_x_p_diff_y_p_num_str=", f_diff_x_p_diff_y_p_num_str)                                                                          
            exec(preparse(f_diff_x_p_diff_y_p_num_str))
            R_p_R_p = (-1)^(p+1)*num_int(lambda y : \
                               (-1)^(p+1)*num_int(lambda x : f_diff_x_p_diff_y_p_num(x,y)  \
                                                              * P(x=x,p=p)/fact(n=p), \
                                 a_x, b_x, v=x)  \
                                                              * P(x=y,p=p)/fact(n=p), \
                                 a_y, b_y, v=y)
        
    return R_p_R_p

In [11]:
#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def euler_maclaurin_R_p_euler_maclaurin_R_p_num_int(f_diff_x_p_diff_y_p, x, a_x, b_x, y, a_y, b_y, p):
    integrand = lambda x, y, p : f_diff_x_p_diff_y_p(x=x,y=y) * P(x=x,p=p)/fact(n=p)
    if logging:
        print("integrand =", integrand(x,y,p))
    R_p_R_p = (-1)^(p+1)*num_int(lambda y : \
              (-1)^(p+1)*num_int(lambda x : integrand(x,y,p), a_x, b_x, x) * P(x=y,p=p)/fact(n=p), a_y, b_y, y)
    return R_p_R_p

In [12]:
def euler_maclaurin1(f, symb, a, b, p, f_diff_symb_p=None, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        print("")
    #u, j = var("u, j")

    int_fn_a_b = integral(f, (symb, a, b))
    if logging:
        print("int_fn_a_b", int_fn_a_b)
        
    half = half_summa(f,symb,a,b)
    if logging:
        print("half", half)
    
    #s = int_fn_a_b + sum_dfdx_a_bernoullis(b) - sum_dfdx_a_bernoullis(a) + R_p
    s = int_fn_a_b + half + sum_dfdx_bernoulis(f,symb,a,b,p)
    
    if True == calc_R_p:
        R_p = euler_maclaurin_R_p(f, symb, a, b, p, f_diff_symb_p=f_diff_symb_p,hold_int=hold_int, hold_sum=hold_sum)
        if type(R_p) is tuple:
            s += R_p[0]
        else:
            s += R_p

    return s

In [13]:
from IPython.display import display, Math, Latex

In [14]:
a_x = var("a_x")
b_x = var("b_x")
a_y = var("a_y")
b_y = var("b_y")

assume(a_x, 'real')
assume(b_x, 'real')
assume(a_y, 'real')
assume(b_y, 'real')

assume(a_x, 'integer')
assume(b_x, 'integer')
assume(a_y, 'integer')
assume(b_y, 'integer')

assume(a_x > 0)
assume(b_x > 0)
assume(a_y > 0)
assume(b_y > 0)

n_x,n_y = var("n_x,n_y")

n = QQ['n'].0
j = QQ['j'].0

p = 4
# p = 3
# p = 2
p = 1

F = function("F")(n_x, n_y)
print(F)

F(n_x, n_y)


$${\displaystyle R_{p}=(-1)^{p+1}\sum_{j=a}^{b-1} \int _{0}^{1}f^{(p)}(u+j){\frac {B_{p}(u)}{p!}}\,du.}$$

In [15]:
use_int_R_p_j=True
change_R_p_order=True
use_int_R_p_shifted_first=True
use_num_int = False
logging = False

In [16]:
int_int = integral(integral           (F,(n_x, a_x, b_x)),  (n_y, a_y, b_y))
print("int_int=", int_int)
display(Math(latex(int_int)))

int_int= integrate(integrate(F(n_x, n_y), n_x, a_x, b_x), n_y, a_y, b_y)


<IPython.core.display.Math object>

In [17]:
int_half = integral (half_summa (F, n_x, a_x, b_x), (n_y, a_y, b_y))
print("int_half=", int_half)
display(Math(latex(int_half)))

int_half= 1/2*integrate(F(a_x, n_y), n_y, a_y, b_y)


<IPython.core.display.Math object>

In [18]:
int_sum = integral(sum_dfdx_bernoulis (F, n_x, a_x, b_x,p), (n_y, a_y, b_y))
print("int_sum=", int_sum)
display(Math(latex(int_sum.expand())))

int_sum= 0


<IPython.core.display.Math object>

In [19]:
R_px = euler_maclaurin_R_p(F, n_x, a_x, b_x,p)
if type(R_px) is tuple:
    inty_R_px = integral(R_px[0], (n_y, a_y, b_y), hold=True)
else:
    inty_R_px = integral(R_px, (n_y, a_y, b_y), hold=True)

print("inty_R_px=",inty_R_px)
display(Math(latex(inty_R_px)))

inty_R_px = integral_euler_maclaurin_R_p(F, n_x, a_x, b_x, n_y, a_y, b_y, p, diff(F,n_x,p), hold_int=True, hold_sum=True)
print("inty_R_px=",inty_R_px)
display(Math(latex(inty_R_px)))


inty_R_px= integrate(integrate(sum(1/2*(2*v_n_x - 1)*D[0](F)(j_n_x + v_n_x, n_y), j_n_x, a_x, b_x - 1), v_n_x, 0, 1), n_y, a_y, b_y)


<IPython.core.display.Math object>

inty_R_px= integrate(integrate(sum(1/2*(2*v_n_x - 1)*D[0](F)(j_x + v_n_x, n_y), j_x, a_x, b_x - 1), v_n_x, 0, 1), n_y, a_y, b_y)


<IPython.core.display.Math object>

In [20]:
half_int = half_summa(integral           (F,(n_x, a_x, b_x)),  n_y, a_y, b_y)
print("half_int=", half_int)
display(Math(latex(half_int)))

half_int= 1/2*integrate(F(n_x, a_y), n_x, a_x, b_x)


<IPython.core.display.Math object>

In [21]:
half_half = half_summa(half_summa(F, n_x, a_x, b_x), n_y, a_y, b_y)
print("half_half=", half_half)
display(Math(latex(half_half)))

half_half= 1/4*F(a_x, a_y)


<IPython.core.display.Math object>

In [22]:
half_sum = half_summa(sum_dfdx_bernoulis (F, n_x, a_x, b_x,p), n_y, a_y, b_y)
print("half_sum=",half_sum)
display(Math(latex(half_sum)))

half_sum= 0


<IPython.core.display.Math object>

In [23]:
use_int_R_p_j=False

# Sagemath 9.3 has bug diff of symbolic_sum
# https://trac.sagemath.org/ticket/32161

R_p = euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p))
if type(R_p) is tuple:
    (R_p, term) = R_p
    print("term=", term)
    display(Math(latex(term)))

half_R_p = half_summa(R_p, n_y, a_y, b_y)
print("half_R_px=",half_R_p)
display(Math(latex(half_R_p)))

half_R_px= -1/4*integrate(-2*n_x*diff(F(n_x, a_y), n_x), n_x, a_x, b_x) - 1/4*integrate(2*floor(n_x)*diff(F(n_x, a_y), n_x), n_x, a_x, b_x) - 1/4*integrate(diff(F(n_x, a_y), n_x), n_x, a_x, b_x)


<IPython.core.display.Math object>

In [24]:
sum_int = sum_dfdx_bernoulis(integral           (F,(n_x, a_x, b_x)),  n_y, a_y, b_y, p)
print("sum_int=",sum_int)
display(Math(latex(sum_int)))

sum_int= 0


<IPython.core.display.Math object>

In [25]:
sum_half = sum_dfdx_bernoulis(half_summa (F, n_x, a_x, b_x), n_y, a_y, b_y, p)
print("sum_half=", sum_half)
display(Math(latex(sum_half)))

sum_half= 0


<IPython.core.display.Math object>

In [26]:
sum_sum = sum_dfdx_bernoulis(sum_dfdx_bernoulis (F, n_x, a_x, b_x,p), n_y, a_y, b_y, p)
print("sum_sum=",sum_sum)
display(Math(latex(sum_sum)))

sum_sum= 0


<IPython.core.display.Math object>

In [27]:
use_int_R_p_j=True
try:
    sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p)), n_y, a_y, b_y, p)
    print("sum_R_p=",sum_R_p)
    display(Math(latex(sum_R_p)))
except Exception as ex:
    print(ex)

sum_R_p= 0


<IPython.core.display.Math object>

In [28]:
use_int_R_p_j=True
try:
    # Sagemath 9.3 has bug diff of symbolic_sum
    # https://trac.sagemath.org/ticket/32161

    R_px = euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p))
    if type(R_px) is tuple:
        (R_px, term) = R_px
        print("term=", term)
        display(Math(latex(term)))


    sumy_R_px = sum_dfdx_bernoulis(R_px, n_y, a_y, b_y, p)
    print("sumy_R_px=",sumy_R_px)
    display(Math(latex(sumy_R_px)))
except Exception as ex:
    print(ex)

sumy_R_px= 0


<IPython.core.display.Math object>

In [29]:
try:
    use_int_R_p_j=True
    sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p)), n_y, a_y, b_y, p)
    print("sum_R_p=",sum_R_p)
    display(Math(latex(sum_R_p)))
except Exception as ex:
    print(ex)

sum_R_p= 0


<IPython.core.display.Math object>

In [30]:
try:
    use_int_R_p_j=True
    # Sagemath 9.3 has bug diff of symbolic_sum
    # https://trac.sagemath.org/ticket/32161

    R_px = euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p))
    print("R_px=", R_px)
    display(Math(latex(R_px)))

    sumy_R_px = sum_dfdx_bernoulis(R_px, n_y, a_y, b_y, p)
    print("sumy_R_px=",sumy_R_px)
    display(Math(latex(sumy_R_px)))

    sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p)), n_y, a_y, b_y, p)
    print("sum_R_p=",sum_R_p)
    display(Math(latex(sum_R_p)))
except Exception as ex:
    print(ex)

R_px= integrate(sum(1/2*(2*v_n_x - 1)*D[0](F)(j_n_x + v_n_x, n_y), j_n_x, a_x, b_x - 1), v_n_x, 0, 1)


<IPython.core.display.Math object>

sumy_R_px= 0


<IPython.core.display.Math object>

sum_R_p= 0


<IPython.core.display.Math object>

In [31]:
use_int_R_p_j=False

R_px = euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p))
print("R_px=", R_px)

sumy_R_px = sum_dfdx_bernoulis(R_px, n_y, a_y, b_y, p)
print("sumy_R_px=",sumy_R_px)
display(Math(latex(sumy_R_px)))


R_px= integrate(1/2*(2*n_x - 2*floor(n_x) - 1)*diff(F(n_x, n_y), n_x), n_x, a_x, b_x)
sumy_R_px= 0


<IPython.core.display.Math object>

In [32]:
int_F = integral           (F,(n_x, a_x, b_x), hold=True)
print("int_F=",int_F)
display(Math(latex(int_F)))

int_F_diff_y_p = int_F.diff(n_y,p)
print("int_F_diff_y_p=",int_F_diff_y_p)
display(Math(latex(int_F_diff_y_p)))

R_p_int = euler_maclaurin_R_p(int_F, n_y, a_y, b_y, p, f_diff_symb_p=int_F_diff_y_p)
print("R_p_int=",R_p_int)
display(Math(latex(R_p_int)))

int_F= integrate(F(n_x, n_y), n_x, a_x, b_x)


<IPython.core.display.Math object>

int_F_diff_y_p= integrate(diff(F(n_x, n_y), n_y), n_x, a_x, b_x)


<IPython.core.display.Math object>

R_p_int= integrate(1/2*(2*n_y - 2*floor(n_y) - 1)*integrate(diff(F(n_x, n_y), n_y), n_x, a_x, b_x), n_y, a_y, b_y)


<IPython.core.display.Math object>

In [33]:
half_F = half_summa (F, n_x, a_x, b_x)
print("half_F=",half_F)
display(Math(latex(half_F)))

half_F_diff_y_p = half_F.diff(n_y,p)
print("half_F_diff_y_p=",half_F_diff_y_p)
display(Math(latex(half_F_diff_y_p)))

R_p_half = euler_maclaurin_R_p(half_F, n_y, a_y, b_y, p, f_diff_symb_p=half_F_diff_y_p)
print("R_p_half=",R_p_half)
display(Math(latex(R_p_half)))

half_F= 1/2*F(a_x, n_y)


<IPython.core.display.Math object>

half_F_diff_y_p= 1/2*diff(F(a_x, n_y), n_y)


<IPython.core.display.Math object>

R_p_half= integrate(1/4*(2*n_y - 2*floor(n_y) - 1)*diff(F(a_x, n_y), n_y), n_y, a_y, b_y)


<IPython.core.display.Math object>

In [34]:
sum_F = sum_dfdx_bernoulis (F, n_x, a_x, b_x,p)
print("sum_F=",sum_F)
display(Math(latex(sum_F)))

sum_F_diff_y_p = diff(sum_F,n_y,p)
print("sum_F_diff_y_p=",sum_F_diff_y_p)
display(Math(latex(sum_F_diff_y_p)))

R_p_sum = euler_maclaurin_R_p(sum_F, n_y, a_y, b_y, p, f_diff_symb_p=sum_F_diff_y_p)
print("R_p_sum=",R_p_sum)
display(Math(latex(R_p_sum)))

sum_F= 0


<IPython.core.display.Math object>

sum_F_diff_y_p= 0


<IPython.core.display.Math object>

R_p_sum= integrate(0, n_y, a_y, b_y)


<IPython.core.display.Math object>

In [35]:
try:
    use_int_R_p_j = True
    # Sagemath 9.3 had bug diff of symbolic_sum
    # https://trac.sagemath.org/ticket/32161

    R_px = euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p))

    if type(R_px) is tuple:
        R_px = R_px[0]

    print("R_px=",R_px)
    display(Math(latex(R_px)))

    # https://trac.sagemath.org/ticket/32394
    # Can't calculate derivative exception while sympifying of two arguments function derivative
    R_px_diff_y_p = R_px.diff(n_y,p)
    print("R_px_diff_y_p=", R_px_diff_y_p)
    display(Math(latex(R_px_diff_y_p)))

    R_p_R_p = euler_maclaurin_R_p(R_px, n_y, a_y, b_y, p, f_diff_symb_p=R_px_diff_y_p)

    print("R_p_R_p=",R_p_R_p)
    display(Math(latex(R_p_R_p)))

except Exception as ex:
    print(ex)

R_px= integrate(sum(1/2*(2*v_n_x - 1)*D[0](F)(j_n_x + v_n_x, n_y), j_n_x, a_x, b_x - 1), v_n_x, 0, 1)


<IPython.core.display.Math object>

R_px_diff_y_p= integrate(sum(1/2*(2*v_n_x - 1)*D[0, 1](F)(j_n_x + v_n_x, n_y), j_n_x, a_x, b_x - 1), v_n_x, 0, 1)


<IPython.core.display.Math object>

R_p_R_p= integrate(sum(1/2*(2*v_n_y - 1)*integrate(sum(1/2*(2*v_n_x - 1)*D[0, 1](F)(j_n_x + v_n_x, j_n_y + v_n_y), j_n_x, a_x, b_x - 1), v_n_x, 0, 1), j_n_y, a_y, b_y - 1), v_n_y, 0, 1)


<IPython.core.display.Math object>

In [36]:
try:
    use_int_R_p_j = False
        
    R_px = euler_maclaurin_R_p(F, n_x, a_x, b_x,p, f_diff_symb_p=F.diff(n_x,p))
    print("R_px=",R_px)
    display(Math(latex(R_px)))

    R_px_diff_y_p = R_px.diff(n_y,p)
    print("R_px_diff_y_p=", R_px_diff_y_p)
    display(Math(latex(R_px_diff_y_p)))

    R_p_R_p = euler_maclaurin_R_p(R_px, n_y, a_y, b_y, p, f_diff_symb_p=R_px_diff_y_p)

    print("R_p_R_p=",R_p_R_p)
    display(Math(latex(R_p_R_p)))

except Exception as ex:
    print(ex)

R_px= integrate(1/2*(2*n_x - 2*floor(n_x) - 1)*diff(F(n_x, n_y), n_x), n_x, a_x, b_x)


<IPython.core.display.Math object>

R_px_diff_y_p= 1/2*integrate((2*n_x - 2*floor(n_x) - 1)*diff(F(n_x, n_y), n_x, n_y), n_x, a_x, b_x)


<IPython.core.display.Math object>

R_p_R_p= integrate(1/4*(2*n_y - 2*floor(n_y) - 1)*integrate((2*n_x - 2*floor(n_x) - 1)*diff(F(n_x, n_y), n_x, n_y), n_x, a_x, b_x), n_y, a_y, b_y)


<IPython.core.display.Math object>

In [37]:
try:
    R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(F, n_x, a_x, b_x,    n_y, a_y, b_y, p, f_diff_x_p_diff_y_p=None)

    print("R_p_R_p=",R_p_R_p)
    display(Math(latex(R_p_R_p)))
except Exception as ex:
    print(ex)

R_p_R_p= integrate(1/4*(2*n_y - 2*floor(n_y) - 1)*integrate((2*n_x - 2*floor(n_x) - 1)*diff(F(n_x, n_y), n_x, n_y), n_x, a_x, b_x), n_y, a_y, b_y)


<IPython.core.display.Math object>

In [38]:
sxy_symb = euler_maclaurin1(euler_maclaurin1(F,n_x,a_x,b_x,p,f_diff_symb_p=None), \
                            n_y,a_y,b_y,p,f_diff_symb_p=None)

In [39]:
display(Math(latex(sxy_symb)))

<IPython.core.display.Math object>

In [40]:
f(n_x,n_y)=1/(n_x^2)
assume(n_y>0)
assume(n_x>0)
a_x = 1
b_x = Infinity

a_y = 1
b_y = Infinity

In [41]:
calc_R_p=True
use_int_R_p_j=False
use_int_R_p_shifted_first=True
change_R_p_order=True

In [42]:
ans = my_euler_maclaurin1(f(n_x, n_y), n_x, a_x, b_x, p, f_diff_symb_p=None)
print(ans)
e2n(ans)

integrate(-(2*n_x - 2*floor(n_x) - 1)/n_x^3, n_x, 1, +Infinity) + 3/2


1.64489926189016

In [43]:
ans = euler_maclaurin1(f(n_x, n_y), n_x, a_x, b_x, p, f_diff_symb_p=None)
print(ans)
ans.n()

integrate(-(2*n_x - 2*floor(n_x) - 1)/n_x^3, n_x, 1, +Infinity) + 3/2


1.64489926189016

In [44]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=False

In [45]:
ans = my_euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None)
print(ans)
ans.n()

1/6*pi^2


1.64493406684823

In [46]:
ans = euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None)
print(ans)
#ans.n()

sum(integrate(-(2*v_n_x - 1)/(j_n_x + v_n_x)^3, v_n_x, 0, 1), j_n_x, 1, +Infinity) + 3/2


In [47]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=True

In [48]:
ans = my_euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None)
print(ans)
ans.n()

integrate(2*v*(harmonic_number(v, 3) - zeta(3)) - harmonic_number(v, 3) + zeta(3), v, 0, 1) + 3/2


1.64493406684823

In [49]:
ans = euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None)
print(ans)
#ans.n()

integrate(sum(-(2*v_n_x - 1)/(j_n_x + v_n_x)^3, j_n_x, 1, +Infinity), v_n_x, 0, 1) + 3/2


In [50]:
ans = euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None, hold_int=False)
print(ans)
#ans.n()

integrate(sum(-(2*v_n_x - 1)/(j_n_x + v_n_x)^3, j_n_x, 1, +Infinity), v_n_x, 0, 1) + 3/2


In [51]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=True
change_R_p_order=False

In [52]:
ans = my_euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None)
print(ans)
ans.n()

1/6*pi^2


1.64493406684823

In [53]:
ans = euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None)
print(ans)
#ans.n()

sum(integrate(-(2*v_n_x - 1)/(j_n_x + v_n_x)^3, v_n_x, 0, 1), j_n_x, 1, +Infinity) + 3/2


In [54]:
ans = euler_maclaurin1(f(n_x, n_y),n_x,a_x,b_x,p, f_diff_symb_p=None, hold_int=False)
print(ans)
#ans.n()

sum(1/2*(2*j_n_x + 3)/(j_n_x^2 + 2*j_n_x + 1) - 1/2*(2*j_n_x - 1)/j_n_x^2, j_n_x, 1, +Infinity) + 3/2


In [55]:
f(n_x,n_y)=1/((n_x^2+n_y^2)^2)

assume(n_y>0)
assume(n_x>0)

a_x = 1
b_x = Infinity

a_y = 1
b_y = Infinity

In [56]:
calc_R_p=True
use_int_R_p_j=True
use_int_R_p_shifted_first=False
change_R_p_order=False

In [57]:
use_int_R_p_j = False
#use_num_int = False

int_int = integral(integral           (f,(n_x, a_x, b_x)),  (n_y, a_y, b_y)) 
print("int_int=", int_int.n())
display(Math(latex(int_int)))

int_half = integral(half_summa(f(n_x, n_y), n_x, a_x, b_x), (n_y, a_y, b_y)) 
print("int_half=", int_half.n())
display(Math(latex(int_half.expand())))

int_sum = integral(sum_dfdx_bernoulis (f(n_x, n_y), n_x, a_x, b_x, p), (n_y, a_y, b_y)) 
print("int_sum=", int_sum.n())
display(Math(latex(int_sum.expand())))

int_int= 0.142699081698724


<IPython.core.display.Math object>

int_half= 0.0713495408493621


<IPython.core.display.Math object>

int_sum= 0.000000000000000


<IPython.core.display.Math object>

In [58]:
try:
    use_num_int = False
    use_int_R_p_j = True
    R_p = euler_maclaurin_R_p(f(n_x, n_y), n_x, a_x, b_x, p, f_diff_symb_p=None)

    print("R_p=", R_p)
    display(Math(latex(R_p)))

    if type(R_p) is tuple:
        int_R_p = integral(R_p[0], (n_y, a_y, b_y), hold=True)
    else:
        int_R_p = integral(R_p, (n_y, a_y, b_y), hold=True)

    print("int_R_p=", int_R_p)
    display(Math(latex(int_R_p)))
    display(Math(latex(int_R_p.n())))
    # The function to be integrated depends on 3 variables (j_x, v_x, y), and so cannot be integrated in one dimension.
    # Please fix additional variables with the 'params' argument
except Exception as ex:
    print(ex)

R_p= (sum(integrate(-2*(j_n_x + v_n_x)*(2*v_n_x - 1)/((j_n_x + v_n_x)^2 + n_y^2)^3, v_n_x, 0, 1), j_n_x, 1, +Infinity), integrate(-2*(j_n_x + v_n_x)*(2*v_n_x - 1)/((j_n_x + v_n_x)^2 + n_y^2)^3, v_n_x, 0, 1))


<IPython.core.display.Math object>

int_R_p= integrate(sum(integrate(-2*(j_n_x + v_n_x)*(2*v_n_x - 1)/((j_n_x + v_n_x)^2 + n_y^2)^3, v_n_x, 0, 1), j_n_x, 1, +Infinity), n_y, 1, +Infinity)


<IPython.core.display.Math object>

The function to be integrated depends on 3 variables (j_n_x, n_y, v_n_x), and so cannot be integrated in one dimension. Please fix additional variables with the 'params' argument


In [59]:
use_num_int = True
use_int_R_p_j = False

f_diff_x_p_str = str(f(x, var("y")).diff(x,p))
print("f_diff_x_p_str=", f_diff_x_p_str)
exec(preparse("f_diff_x_p = lambda x, y :" + f_diff_x_p_str))

int_R_p_ans = integral_euler_maclaurin_R_p(f, n_x, a_x, b_x, n_y, a_y, b_y, p, f_diff_x_p=f_diff_x_p, hold_int=False)
if type(int_R_p_ans) is tuple:
    (int_R_p,int_int_R_p) = int_R_p_ans
else:
    int_R_p = int_R_p_ans

print("int_R_p=",int_R_p)
display(Math(latex(int_R_p)))

f_diff_x_p_str= -4*x/(x^2 + y^2)^3


/tmp/ipykernel_1133/2434198373.py:11: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral = integrate.quad(f, a, b)
/tmp/ipykernel_1133/2434198373.py:11: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral = integrate.quad(f, a, b)


int_R_p= 0.014591148758590414


<IPython.core.display.Math object>

In [60]:
int_int

1/8*pi - 1/4

In [61]:
int_int.n()

0.142699081698724

In [62]:
int_sum

0

In [63]:
int_sum.n()

0.000000000000000

In [64]:
int_half

1/16*pi - 1/8

In [65]:
int_half.n()

0.0713495408493621

In [66]:
int_R_p

0.014591148758590414

In [67]:
try:
    print(sum(int_int_R_p, j_x, 1, +Infinity))
except Exception as ex:
    print(ex)

name 'int_int_R_p' is not defined


In [68]:
try:
    int_R_p_s = 0
    for j in range (1, 1000):
        int_R_p_term = int_int_R_p.subs(var("j_x") == j)
        int_R_p_s += int_R_p_term
    int_R_p_n = int_R_p_s.n()
    print(int_R_p_n)
except Exception as ex:
    print(ex)
    int_R_p_n = int_R_p
    print(int_R_p_n)

name 'int_int_R_p' is not defined
0.014591148758590414


In [69]:
half_int = half_summa(integral           (f(n_x,n_y),(n_x, a_x, b_x)),  n_y, a_y, b_y)
print("half_int=",half_int.n())
display(Math(latex(half_int)))

half_half = half_summa(half_summa (f(n_x,n_y), n_x, a_x, b_x), n_y, a_y, b_y)
print("half_half=", half_half.n())
display(Math(latex(half_half)))

half_sum = half_summa(sum_dfdx_bernoulis (f(n_x,n_y), n_x, a_x, b_x, p), n_y, a_y, b_y)
print("half_sum=", half_sum.n())
display(Math(latex(half_sum)))

use_num_int = False
f_diff_symb_p = f(n_x, n_y).diff(n_x, p)
print("f_diff_symb_p=", f_diff_symb_p)
R_p = euler_maclaurin_R_p(f(n_x, n_y), n_x, a_x, b_x, p, f_diff_symb_p=f_diff_symb_p)
half_R_p = half_summa(R_p, n_y, a_y, b_y)

half_R_p_n = half_R_p.n()
print("half_R_p=", half_R_p_n)
display(Math(latex(half_R_p)))

half_int= 0.0713495408493621


<IPython.core.display.Math object>

half_half= 0.0625000000000000


<IPython.core.display.Math object>

half_sum= 0.000000000000000


<IPython.core.display.Math object>

f_diff_symb_p= -4*n_x/(n_x^2 + n_y^2)^3
half_R_p= 0.01956886117074705


<IPython.core.display.Math object>

In [70]:
sum_int = sum_dfdx_bernoulis(integral           (f(n_x,n_y),(n_x, a_x, b_x)),  n_y, a_y, b_y, p)
print("sum_int=", e2n(sum_int))
display(Math(latex(sum_int)))

sum_half = sum_dfdx_bernoulis(half_summa (f(n_x,n_y), n_x, a_x, b_x), n_y, a_y, b_y, p)
print("sum_half=", e2n(sum_half))
display(Math(latex(sum_half)))

sum_sum = sum_dfdx_bernoulis(sum_dfdx_bernoulis (f(n_x,n_y), n_x, a_x, b_x,p), n_y, a_y, b_y, p)
print("sum_sum=", e2n(sum_sum))
display(Math(latex(sum_sum)))

sum_int= 0


<IPython.core.display.Math object>

sum_half= 0


<IPython.core.display.Math object>

sum_sum= 0


<IPython.core.display.Math object>

In [71]:
use_int_R_p_j=False

R_px = euler_maclaurin_R_p(f(n_x,n_y), n_x, a_x, b_x, p, f_diff_symb_p=f(n_x,n_y).diff(n_x,p))
print("R_px=", R_px)
display(Math(latex(R_px)))

sumy_R_px = sum_dfdx_bernoulis(R_px, n_y, a_y, b_y, p)
print("sumy_R_px=",sumy_R_px)
display(Math(latex(sumy_R_px)))

R_px= integrate(-2*(2*n_x - 2*floor(n_x) - 1)*n_x/(n_x^2 + n_y^2)^3, n_x, 1, +Infinity)


<IPython.core.display.Math object>

sumy_R_px= 0


<IPython.core.display.Math object>

In [72]:
sumy_R_px_n = e2n(sumy_R_px)
sumy_R_px_n

0

In [73]:
if use_int_R_p_j:
    print(dfdx_a_bernoullis_terms(j_x))

In [74]:
if use_int_R_p_j:
    sum_R_p_j_n = 0
    for j_x in range(1,100):
        term = dfdx_a_bernoullis_terms(j_x)
        sum_R_p_j_n += term
        #print(j_x, term.n(), sum_R_p_n)
    print(sum_R_p_j_n.n())

In [75]:
use_int_R_p_j = False
use_num_int = False
_int = integral           (f,(n_x, a_x, b_x), hold = False)
print("_int=", _int)
R_p_int = euler_maclaurin_R_p(_int,  n_y, a_y, b_y, p, f_diff_symb_p=None)
print("R_p_int=", R_p_int)
display(Math(latex(R_p_int)))

_int= n_y |--> -1/2*(n_y^2*arctan(1/n_y) + n_y + arctan(1/n_y))/(n_y^5 + n_y^3) + 1/4*pi/n_y^3
R_p_int= integrate(1/8*(2*n_y - 2*floor(n_y) - 1)*(2*(5*n_y^4 + 3*n_y^2)*(n_y^2*arctan(1/n_y) + n_y + arctan(1/n_y))/(n_y^5 + n_y^3)^2 - 2*(2*n_y*arctan(1/n_y) - 1/(1/n_y^2 + 1) - 1/(n_y^2*(1/n_y^2 + 1)) + 1)/(n_y^5 + n_y^3) - 3*pi/n_y^4), n_y, 1, +Infinity)


<IPython.core.display.Math object>

In [76]:
R_p_int

integrate(1/8*(2*n_y - 2*floor(n_y) - 1)*(2*(5*n_y^4 + 3*n_y^2)*(n_y^2*arctan(1/n_y) + n_y + arctan(1/n_y))/(n_y^5 + n_y^3)^2 - 2*(2*n_y*arctan(1/n_y) - 1/(1/n_y^2 + 1) - 1/(n_y^2*(1/n_y^2 + 1)) + 1)/(n_y^5 + n_y^3) - 3*pi/n_y^4), n_y, 1, +Infinity)

In [77]:
R_p_int_n = R_p_int.n()
R_p_int_n

0.014593410520966651

In [78]:
use_int_R_p_j = False
use_num_int = False

R_p_half = euler_maclaurin_R_p(half_summa (f, n_x, a_x, b_x), n_y, a_y, b_y, p, f_diff_symb_p=None, hold_int=False)
print("R_p_half  =", R_p_half )
display(Math(latex(R_p_half)))

R_p_half  = n_x |--> integrate(-(2*n_y - 2*floor(n_y) - 1)*n_y/(n_y^2 + 1)^3, n_y, 1, +Infinity)


<IPython.core.display.Math object>

In [79]:
R_p_half_n = R_p_half.n()
R_p_half_n

0.019569055005268107

In [80]:
use_int_R_p_j = False
use_num_int = False

R_p_sum = euler_maclaurin_R_p(sum_dfdx_bernoulis (f, n_x, a_x, b_x,p), n_y, a_y, b_y, p, f_diff_symb_p=None, hold_int=False)
print("R_p_sum =",R_p_sum)
display(Math(latex(R_p_sum)))

R_p_sum = 0


<IPython.core.display.Math object>

In [81]:
R_p_sum_n = R_p_sum.n()
R_p_sum_n

0.000000000000000

In [82]:
#use_num_int = True

#R_p_R_p = euler_maclaurin_R_p(euler_maclaurin_R_p(f(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
#R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p(f(x,y), x, a_x, b_x,    y, a_y, b_y, p)
#print("R_p_R_p=$", R_p_R_p.n())
#display(Math(latex(R_p_R_p)))

In [83]:
f_diff_x_p_diff_y_p_ = f(x, y).diff(x,p).diff(y,p)
exec(preparse("f_diff_x_p_diff_y_p = lambda x, y:" + str(f_diff_x_p_diff_y_p_)))
R_p_R_p = euler_maclaurin_R_p_euler_maclaurin_R_p_num_int(f_diff_x_p_diff_y_p, n_x, a_x, b_x, n_y, a_y, b_y, p)

/tmp/ipykernel_1133/2434198373.py:11: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral = integrate.quad(f, a, b)
/tmp/ipykernel_1133/2434198373.py:11: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integral = integrate.quad(f, a, b)


In [84]:
R_p_R_p

0.008150899241692044

# Summary Euler–Maclaurin 2D for ${\displaystyle \sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2} }$

In [85]:
int_int, int_half, int_sum, int_R_p

(1/8*pi - 1/4, 1/16*pi - 1/8, 0, 0.014591148758590414)

In [86]:
half_int, half_half, half_sum, half_R_p

(1/16*pi - 1/8,
 1/16,
 0,
 integrate(-2*n_x^2/(n_x^6 + 3*n_x^4 + 3*n_x^2 + 1), n_x, 1, +Infinity) + integrate(2*n_x*floor(n_x)/(n_x^6 + 3*n_x^4 + 3*n_x^2 + 1), n_x, 1, +Infinity) + integrate(n_x/(n_x^6 + 3*n_x^4 + 3*n_x^2 + 1), n_x, 1, +Infinity))

In [87]:
sum_int, sum_half, sum_sum, sumy_R_px

(0, 0, 0, 0)

In [88]:
R_p_int, R_p_half, R_p_sum, R_p_R_p

(integrate(1/8*(2*n_y - 2*floor(n_y) - 1)*(2*(5*n_y^4 + 3*n_y^2)*(n_y^2*arctan(1/n_y) + n_y + arctan(1/n_y))/(n_y^5 + n_y^3)^2 - 2*(2*n_y*arctan(1/n_y) - 1/(1/n_y^2 + 1) - 1/(n_y^2*(1/n_y^2 + 1)) + 1)/(n_y^5 + n_y^3) - 3*pi/n_y^4), n_y, 1, +Infinity),
 n_x |--> integrate(-(2*n_y - 2*floor(n_y) - 1)*n_y/(n_y^2 + 1)^3, n_y, 1, +Infinity),
 0,
 0.008150899241692044)

Numerical Euler–Maclaurin 2D for $\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}$

In [89]:
"%f %f %f %f" % (int_int.n(), int_half.n(), int_sum.n(), int_R_p_n)

'0.142699 0.071350 0.000000 0.014591'

In [90]:
"%f %f %f %f" % (half_int.n(), half_half.n(), half_sum.n(), half_R_p_n)

'0.071350 0.062500 0.000000 0.019569'

In [91]:
"%f %f %f %f" % (e2n(sum_int), e2n(sum_half), e2n(sum_sum), sumy_R_px_n)

'0.000000 0.000000 0.000000 0.000000'

In [92]:
"%f %f %f %f" % (R_p_int_n, R_p_half_n, R_p_sum_n, R_p_R_p)

'0.014593 0.019569 0.000000 0.008151'

In [93]:
int_int.n()  + int_half.n()  + int_sum.n()  + int_R_p_n    + \
half_int.n() + half_half.n() + half_sum.n() + half_R_p.n() + \
e2n(sum_int) + e2n(sum_half) + e2n(sum_sum) + sumy_R_px_n    + \
R_p_int_n    + R_p_half_n    + R_p_sum_n    + R_p_R_p

0.424371538094713

$${\displaystyle \sum \limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} }$$

In [94]:
def zeta_2D_1_1(S, N):
    
    def zeta_2D_row(S, first, N):
        row_sum = 0
        for m in range(first, N):
            row_sum += (N^2+m^2)^(-S)
        return row_sum

    def zeta_2D_diag(S, N):
        diag_elem = (2*N^2)^(-S)
        return diag_elem
    
    zeta_2D_sum = 0
    for n in range(1,N):
        zeta_2D_add = 2 * zeta_2D_row(S, 1, n) + zeta_2D_diag(S, n)
        zeta_2D_sum += zeta_2D_add
        
    return zeta_2D_sum

In [95]:
zeta_2D_1_1(2, 1000)

0.4243791332035987

R_p=False (Euler–Maclaurin 2D without remainer part)

In [96]:
int_int.n()  + int_half.n()  + int_sum.n()  + \
half_int.n() + half_half.n() + half_sum.n() + \
e2n(sum_int) + e2n(sum_half) + e2n(sum_sum)

0.347898163397448

# Two Dimensional Linear Zeta Function

https://core.ac.uk/download/pdf/82162715.pdf

JOURNAL OF NUMBER THEORY 35, 62-71 (1990)

Evaluation of Multidimensional Linear Zeta Functions

ALFRED ACTOR*

Department of Physics, The Pennsylvania State University,

Fogelsville, Pennsylvania 18051

Communicated by Hans Zassenhaus

Received January 28, 1988; revised May 5, 1989

A prototypical formula of this type is [1] 

$\sum \limits_{m,n=-\infty}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = 4 \, \zeta \left(S\right) \, \beta \left(S\right)$

where the prime means that $n = m = 0$ is excluded from the sum. 

where of course, $\zeta\left(S\right)$ is the Riemann $\zeta$-function [2]

while $\beta\left(S\right) = \sum \limits_{n=0}^{\infty}\left( - \right)^n \left(2n + 1\right)^{-S}$
defines, for $Re S > 0$, a function whose properties throughout the s-plane are
well known [3].

Thus Eq. (1) fully reveals the properties of the twodimensional Epstein $\zeta$-function [4] on the left. 



1. G. H. HARDY, Notes on some points in the integral calculus LII, Mess. Math. 49 (1919),
85-91.
2. H. M. EDWARDS, “Riemann’s Zeta Function,” Academic Press, New York, 1974.
3. A. ERDELYI, W. MAGNUS, F. OEERHETTINGER, AND F. G. TRICOMI (Eds.), “Higher Transcendental Functions,” Vol. 1, McGraw-Hill, New York, 1953. 
LINEAR ZETA FUNCTIONS 71
4. P. EPSTEIN, Zur Theorie allgemeiner Zetafunctionen, Marh. Ann. 56 (1903), 615-645; Zur
Theorie allgemeiner Zetafunctionen, II, Math. Ann. 63 (19071, 205-216. 

$${\displaystyle \sum\limits_{m,n=(0)1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S} = \zeta\left(S\right)\,\beta\left(S\right) }$$

$${\displaystyle \sum\limits_{m,n=(0)1}^{\infty} \, \left(\sqrt{m^{2} + n^{2}}\right)^{-S} = \sum\limits_{m,n=(0)1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-\frac{S}{2}} = \zeta\left(\frac{S}{2}\right)\,\beta\left(\frac{S}{2}\right) }$$

In [97]:
def zeta_2D(S, N):
    
    def zeta_2D_row(S, first, N):
        row_sum = 0
        for m in range(first, N):
            row_sum += (N^2+m^2)^(-S)
        return row_sum

    def zeta_2D_diag(S, N):
        diag_elem = (2*N^2)^(-S)
        return diag_elem
    
    zeta_2D_sum = 0
    for n in range(1,N):
        zeta_2D_add = zeta_2D_row(S, 0, n) + zeta_2D_row(S, 1, n) + zeta_2D_diag(S, n)
        zeta_2D_sum += zeta_2D_add
        
    return zeta_2D_sum

$${\displaystyle \beta\left(S\right) = \sum\limits_{n=0}^{\infty}\left( - \right)^n \left(2n + 1\right)^{-S} }$$

$${\displaystyle \beta\left(S\right) = \sum\limits_{n=0}^{\infty}\left( - \right)^{2n} \left(2\cdot2n + 1\right)^{-S} +\left( - \right)^{(2n+1)} \left(2(2n+1) + 1\right)^{-S} }$$

$${\displaystyle \beta\left(S\right) = \sum\limits_{n=0}^{\infty}\left( - \right)^{2n} \left(4n + 1\right)^{-S} +\left( - \right)^{(2n+1)} \left(4n+3\right)^{-S} }$$

$${\displaystyle \beta\left(S\right) = \sum\limits_{n=0}^{\infty} \left(4n + 1\right)^{-S} - \left(4n+3\right)^{-S} }$$

$${\displaystyle \beta\left(S\right) = \sum\limits_{n=0}^{\infty} \left(4n + 1\right)^{-S} - \sum\limits_{n=0}^{\infty}\left(4n+3\right)^{-S} }$$

$${\displaystyle \beta\left(S\right) = 4^{-S}\sum\limits_{n=0}^{\infty} \left(n + \frac{1}{4}\right)^{-S} - 4^{-S}\sum\limits_{n=0}^{\infty}\left(n+\frac{3}{4}\right)^{-S} }$$

In [98]:
from sage.calculus.calculus import symbolic_sum
n = var("n")
S = var("S")
beta1_sym = lambda S : symbolic_sum((-1)^n * (2*n + 1)^(-S), n, 0, Infinity)
beta1 = lambda S : sum((-1)^n * (2*n + 1)^(-S), n, 0, Infinity)

def beta1_dig(S,N):
    Sum = 0
    for n in range (N):
        Sum += (-1)^n * (2*n + 1)^(-S)
    return Sum

In [99]:
def beta2_dig(S,N):
    Sum = 0
    for n in range (N/2):
        Sum += (-1)^(2*n)   * (2*(2*n)   + 1)^(-S)
        Sum += (-1)^(2*n+1) * (2*(2*n+1) + 1)^(-S)
    return Sum

$${\displaystyle \beta\left(S\right) = \sum\limits_{n=0}^{\infty} \left(4n + 1\right)^{-S} - \sum\limits_{n=0}^{\infty}\left(4n+3\right)^{-S} }$$

In [100]:
def beta2_dig_1(S,N):
    Sum = 0
    for n in range (N/2):
        Sum += (-1)^(2*n)   * (2*(2*n)   + 1)^(-S)
    return Sum

In [101]:
def beta2_dig_2(S,N):
    Sum = 0
    for n in range (N/2):
        Sum += (-1)^(2*n+1) * (2*(2*n+1) + 1)^(-S)
    return Sum

In [102]:
(beta2_dig_1(2,100000)/zeta(2)).n()

0.653419394620402

In [103]:
(beta2_dig_2(2,100000)/zeta(2)).n()

-0.0965790855618431

In [104]:
def beta3_dig(S,N):
    Sum = 0
    for n in range (N/2):
        Sum += (4*n + 1)^(-S) - (4*n+3)^(-S)
    return Sum

In [105]:
def beta4_dig(S,N):
    Sum = 0
    for n in range (N/2):
        Sum += (n + 1/4)^(-S) - (n+3/4)^(-S)
    return Sum*4^(-S)

$${\displaystyle \sum\limits_{m,n=0,1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}=\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-S}+\zeta(2 S) }$$

$${\displaystyle \sum\limits_{m,n=0,1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}=\sum\limits_{m,n=1}^{\infty} \, \left(m^{2} + n^{2}\right)^{-2}+\zeta(4) }$$

In [106]:
ans_euler_maclarin_2D = \
int_int.n()  + int_half.n()  + int_sum.n()  + int_R_p_n    + \
half_int.n() + half_half.n() + half_sum.n() + half_R_p.n() + \
e2n(sum_int) + e2n(sum_half) + e2n(sum_sum) + sumy_R_px_n  + \
R_p_int_n    + R_p_half_n    + R_p_sum_n    + R_p_R_p      + \
zeta(4).n()
ans_euler_maclarin_2D

1.50669477180585

Because resulting Matrix has some difference from simmetric one due difference between R_p_half and half_R_p , I will do some test

In [107]:
ans_euler_maclarin_2D_half_R_p = \
int_int.n()  + int_half.n()  + int_sum.n()  + int_R_p_n    + \
half_int.n() + half_half.n() + half_sum.n() + half_R_p.n() + \
e2n(sum_int) + e2n(sum_half) + e2n(sum_sum) + sumy_R_px_n  + \
R_p_int_n    + half_R_p.n()  + R_p_sum_n    + R_p_R_p      + \
zeta(4).n()
ans_euler_maclarin_2D_half_R_p

1.50669457797133

In [108]:
ans_euler_maclarin_2D_R_p_half = \
int_int.n()  + int_half.n()  + int_sum.n()  + int_R_p_n   + \
half_int.n() + half_half.n() + half_sum.n() + R_p_half_n  + \
e2n(sum_int) + e2n(sum_half) + e2n(sum_sum) + sumy_R_px_n + \
R_p_int_n    + R_p_half_n    + R_p_sum_n    + R_p_R_p     + \
zeta(4).n()
ans_euler_maclarin_2D_R_p_half

1.50669496564037

$${\displaystyle \frac{1}{4}\sum \limits_{m,n=-\infty}^{\infty} \left(m^{2} + n^{2}\right)^{-S} = \zeta \left(S\right)\beta\left(S\right) }$$

In [109]:
beta1_dig(2,1000)*zeta(2).n()

1.50670280430638

In [110]:
beta2_dig(2,1000)*zeta(2).n()

1.50670280430638

In [111]:
beta3_dig(2,1000)*zeta(2).n()

1.50670280430638

In [112]:
beta3_dig(2,1000)*zeta(2).n()

1.50670280430638

In [113]:
beta4_dig(2,1000)*zeta(2).n()

1.50670280430638

In [114]:
zeta_2D(2, 1000)

1.5067023665809038

In [115]:
zeta_2D(2, 1000) - ans_euler_maclarin_2D

7.59477505352102e-6

In [116]:
zeta_2D(2, 1000) - ans_euler_maclarin_2D_half_R_p

7.78860957439953e-6

In [117]:
zeta_2D(2, 1000) - ans_euler_maclarin_2D_R_p_half

7.40094053219842e-6

So I see that R_p_half is more accurate than half_R_p